In [3]:
def parse_input(is_test=True):
    file_name = "input-test.txt" if is_test else "input.txt"
    with open(file_name, "r") as file:
        line = file.readline()
        return [int(c) for c in list(line.strip())]

# Part 1


In [4]:
def generate_index_value_mapping(input: str) -> dict:
    free_space_annonation = "."

    result = dict()
    current_index = -1
    for i in range(len(input)):
        size = input[i]
        is_free_space = i % 2 != 0
        file_id = i // 2
        for _ in range(size):
            result[current_index + 1] = free_space_annonation if is_free_space else file_id
            current_index += 1

    return result

In [5]:
def find_first_free_space_key(index_value_mapping: dict) -> int:
    sorted_keys = sorted(list(index_value_mapping.keys()))
    for key in sorted_keys:
        if index_value_mapping[key] == ".":
            return key

    return sorted_keys[-1] + 1

In [6]:
def find_last_file_block_key(index_value_mapping: dict) -> int:
    reverse_sorted_keys = sorted(list(index_value_mapping.keys()))[::-1]
    for key in reverse_sorted_keys:
        if index_value_mapping[key] != ".":
            return key

    return reverse_sorted_keys[0] - 1

In [7]:
# sliding windows with 2 pointers
def file_compact(index_value_mapping: dict) -> None:
    left = find_first_free_space_key(index_value_mapping)
    right = find_last_file_block_key(index_value_mapping)
    while left < right:
        # left is . and right is X
        if index_value_mapping[left] == "." and index_value_mapping[right] != ".":
            index_value_mapping[left], index_value_mapping[right] = (
                index_value_mapping[right],
                index_value_mapping[left],
            )
            left += 1
            right -= 1
        # left is X and right is X
        elif index_value_mapping[left] != "." and index_value_mapping[right]:
            left += 1
        # left is . and right is .
        elif index_value_mapping[left] == "." and index_value_mapping[right] == ".":
            right -= 1
        # left is X and right is .
        else:
            left += 1
            right -= 1

In [8]:
def check_sum(index_value_mapping: dict) -> int:
    result = 0
    for key, value in index_value_mapping.items():
        if value == ".":
            result += 0
            continue
        result += key * value

    return result

In [9]:
def solve_part_1():
    input = parse_input(is_test=False)
    index_value_mapping = generate_index_value_mapping(input)
    file_compact(index_value_mapping)
    result = check_sum(index_value_mapping)

    print(f"filesystem checksum for part 1 is {result}")

In [10]:
solve_part_1()

filesystem checksum for part 1 is 6356833654075


# Part 2


In [11]:
def generate_file_blocks_and_free_spaces_mapping(input: str) -> dict:
    result_file_blocks = dict()
    result_free_spaces = list()
    current_index = 0
    for i in range(len(input)):
        size = input[i]
        if size == 0:
            continue

        is_free_space = i % 2 != 0
        file_id = i // 2

        # free spaces
        if is_free_space:
            free_space_indexes = [current_index + j for j in range(size)]
            result_free_spaces.append(free_space_indexes)
            current_index += size
            continue

        # file blocks
        file_block_indexes = [current_index + j for j in range(size)]
        result_file_blocks[file_id] = file_block_indexes
        current_index += size

    return result_file_blocks, result_free_spaces

In [12]:
def rearrange_file(file_blocks: dict, free_spaces: list) -> None:
    reverse_file_blocks_keys = list(file_blocks.keys())[::-1]

    for key in reverse_file_blocks_keys:
        len_free_spaces_needed = len(file_blocks[key])
        for i in range(len(free_spaces)):
            len_free_spaces_available = len(free_spaces[i])
            if len_free_spaces_available >= len_free_spaces_needed and max(free_spaces[i]) < min(file_blocks[key]):
                file_blocks[key] = free_spaces[i][:len_free_spaces_needed]
                free_spaces[i] = free_spaces[i][len_free_spaces_needed:]
                break

In [13]:
def check_sum_2(file_blocks: dict):
    result = 0
    for key, value in file_blocks.items():
        result += key * sum(value)

    return result

In [14]:
def solve_part_2():
    input = parse_input(is_test=False)
    file_blocks, free_spaces = generate_file_blocks_and_free_spaces_mapping(input)
    rearrange_file(file_blocks, free_spaces)
    result = check_sum_2(file_blocks)

    print(f"filesystem checksum for part 2 is {result}")


solve_part_2()

filesystem checksum for part 2 is 6389911791746
